In [1]:
!pip install keras_tuner -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.6/169.6 KB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 30.6 MB/s eta 0:00:00


In [2]:
import OlegFunction2023 as OF23
import os

from tensorflow.keras import models
import keras_tuner as kt
from keras_tuner import RandomSearch, Hyperband, BayesianOptimization

# !pip install keras_tuner -q
# from tensorflow.keras import models
# import keras_tuner as kt
# from keras_tuner import RandomSearch, Hyperband, BayesianOptimization

# All_PV = True
All_PV = False
Max_trials = 30
# Max_trials = 10
Executions_per_trial=1
RandomTuner = True
Max_best_model = 5
# Max_best_model = 3

In [ ]:
# import tensorflow as tf0
# sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))

# from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())

# print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

# import tensorflow as tf
# config = tf.compat.v1.ConfigProto( device_count = {'GPU': 1 , 'CPU': 6} ) 
# sess = tf.compat.v1.Session(config=config) 
# from tensorflow.python.keras import backend as K
# K.set_session(sess)

In [3]:
Dnn = OF23.DNN_Parameters()

if All_PV:
    DataReader=OF23.BigPV_DataRead
    Dnn.InputShape=6
else:
    DataReader=OF23.Isc_DataRead 
    Dnn.InputShape=4
DT = OF23.ScalerLogDataTransform()

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator StandardScaler from version 0.23.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [4]:
def DataTestRead(*FileName):
    dataTest = DataReader(*FileName)
    return DT.DataNormalize(dataTest)

In [5]:
data = DataReader("ResultAllAM15.dat")
# data = DataReader("ResultAll940.dat")
# data = DataReader("ResultAllAM15.dat","B_varAM15.dat","Fe_varAM15.dat","All_varAM15.dat")
dataTrain = OF23.SplitInTrainAndTest(data,1)
Xtrain,Ytrain = DT.DataNormalizeTrain(dataTrain)

# XtestBvar,YtestBvar = DataTestRead("B_varAM15.dat")
# XtestFevar,YtestFevar = DataTestRead("Fe_varAM15.dat")
# XtestAllvar,YtestAllvar = DataTestRead("All_varAM15.dat")
# XtestMixt,YtestMixt = DataTestRead("B_varAM15.dat","Fe_varAM15.dat","All_varAM15.dat")

In [6]:
def ModelCreateAndCompileHP(hp):
    Dnn.FirstHLNumber = hp.Choice('FirstHLNumber', values = [5, 10, 30, 50, 75, 100, 120, 150, 200])
    Dnn.LayersNumber = hp.Choice('LayNumber', values = [4, 5 , 6, 8, 10, 12, 15])
    Dnn.SetConfigurationTypeByNumber(hp.Choice('Configuration', values = [1, 2, 3, 4, 5]))
# 1 - 'Pipe'
# 2 - 'Trapezoi'
# 3 - 'Triangle'
# 4 - 'Butterfly'
# 5 - 'Fir'
    
    
    Dnn.BS = hp.Choice('BatchSize', values = [8, 16, 32, 64, 128])
#     Dnn.Epoch = hp.Choice('Epotchs', values = [100, 300, 400, 600, 1000, 1500]) 
    Dnn.Epoch = hp.Choice('Epotchs', values = [100]) 

    Dnn.LRate = hp.Float("LR", min_value=1e-5, max_value=1e-2, sampling="log")
    Dnn.SetOptimizerFunByNumber(hp.Choice('Optim', values = [0, 1, 2 , 3, 4, 5, 6, 7]))
# 0 'Adam'
# 1 'SGD'
# 2 'RMSprop'
# 3 'Adadelta'
# 4 'Adagrad'
# 5 'Adamax'
# 6 'Nadam'
# 7 'Ftrl'

    Dnn.SetActivationFunByNumber(hp.Choice('Activat', values = [0, 1, 2 , 3, 4]))
# 0 'relu'
# 1 'sigmoid'
# 2 'tanh'
# 3 'selu'
# 4 'elu'

    Dnn.SetInitializerTypeByNumber(hp.Choice('Init', values = [0, 1, 2 , 3, 4, 5]))
# 0 'XavierNormal'
# 1 'XavierUniform'
# 2 'HeNormal'
# 3 'HeUniform'
# 4 'RandomNormal'
# 5 'RandomUniform'


#     Dnn.RegularizerTypeByNumber(hp.Choice('Regul', values = [0, 1, 2]))
    Dnn.RegularizerTypeByNumber(hp.Choice('Regul', values = [0]))
# 0 'None'
# 1 'L1'
# 2 'L2'
    if Dnn.RegularizersIndex != 0:
        Dnn.RegRate = hp.Choice('RR', values = [1e-5, 1e-4, 1e-3, 1e-2])
    
    Dnn.DropoutNeeded = hp.Choice('DropNeed', values = [False])    
#     Dnn.DropoutNeeded = hp.Boolean("DropNeed")
    if Dnn.DropoutNeeded:
        Dnn.DropRate = hp.Choice('DropR', values = [0.2, 0.3, 0.4, 0.5])
    
    
    return Dnn.ModelCreateAndCompile()

In [7]:
if RandomTuner:
    tuner = RandomSearch(
        ModelCreateAndCompileHP, 
        objective = 'val_loss', 
        max_trials=Max_trials, 
        executions_per_trial=Executions_per_trial,
        directory = 'test_directory',
        project_name='Random',
        overwrite=True
        )
else:
    tuner = BayesianOptimization(
        ModelCreateAndCompileHP, 
        objective = 'val_loss', 
        max_trials=Max_trials, 
        executions_per_trial=Executions_per_trial,
        directory = 'test_directory',
        project_name='Bayes',
        overwrite=True,
    )

In [8]:
tuner.search(Xtrain, 
         Ytrain, 
         batch_size = Dnn.BS, 
         epochs = Dnn.Epoch,           
         validation_split = 0.2, 
         verbose = 1,
         )

Trial 30 Complete [00h 08m 51s]
val_loss: 1.0072340965270996

Best val_loss So Far: 0.03836469724774361
Total elapsed time: 04h 33m 56s


In [9]:
def BestParametersToDNNandPrint(best_hp,HeaderText):
    print(HeaderText)
    Dnn.FirstHLNumber=best_hp.get('FirstHLNumber')
    Dnn.LayersNumber=best_hp.get('LayNumber')
    Dnn.BS = best_hp.get('BatchSize')
    Dnn.LRate = best_hp.get('LR')
    Dnn.Epoch = best_hp.get('Epotchs')
    Dnn.SetConfigurationTypeByNumber(best_hp.get('Configuration'))
    Dnn.SetOptimizerFunByNumber(best_hp.get('Optim'))
    Dnn.SetActivationFunByNumber(best_hp.get('Activat'))
    Dnn.SetInitializerTypeByNumber(best_hp.get('Init'))
    Dnn.RegularizerTypeByNumber(best_hp.get('Regul'))
    if Dnn.RegularizersIndex != 0:
        Dnn.RegRate = best_hp.get('RR')
    Dnn.DropoutNeeded=best_hp.get('DropNeed')
    if Dnn.DropoutNeeded:
        Dnn.DropRate = best_hp.get('DropR') 
    print(Dnn)    

In [11]:
#%%script false
best_hyperparameters = tuner.get_best_hyperparameters(Max_best_model)
for best_hyperparameter in best_hyperparameters:
    BestParametersToDNNandPrint(best_hyperparameter,
                                'Model rating: %02d' %(best_hyperparameters.index(best_hyperparameter)))
    print('---------------------------------------------------------')

Model rating: 00
#Input Nodes = 4
ConfigurationType - Butterfly
[75, 67, 59, 51, 43, 35, 35, 43, 51, 59, 67, 75]
Batch Size = 8
# Epochs = 100
Learning Rate = 5.2712507092710046e-05
Ortimizer = Adam
Activation = elu
Initializer = XavierNormal
Regularizer = None
Dropout is not used
---------------------------------------------------------
Model rating: 01
#Input Nodes = 4
ConfigurationType - Triangle
[10, 9, 8, 7, 6, 5, 4, 3, 2, 1]
Batch Size = 16
# Epochs = 100
Learning Rate = 0.0032649444211699905
Ortimizer = Adam
Activation = elu
Initializer = HeNormal
Regularizer = None
Dropout is not used
---------------------------------------------------------
Model rating: 02
#Input Nodes = 4
ConfigurationType - Fir
[30, 27, 24, 21, 18, 15, 30, 27, 24, 21, 18, 15]
Batch Size = 128
# Epochs = 100
Learning Rate = 0.002215807058277498
Ortimizer = Nadam
Activation = selu
Initializer = RandomUniform
Regularizer = None
Dropout is not used
---------------------------------------------------------
Model

In [12]:
tuner.results_summary(Max_best_model)

Results summary
Results in test_directory/Random
Showing 5 best trials
Trial summary
Hyperparameters:
FirstHLNumber: 75
LayNumber: 8
Configuration: 4
BatchSize: 8
Epotchs: 100
LR: 5.2712507092710046e-05
Optim: 0
Activat: 4
Init: 0
Regul: 0
DropNeed: 0
Score: 0.03836469724774361
Trial summary
Hyperparameters:
FirstHLNumber: 10
LayNumber: 15
Configuration: 3
BatchSize: 16
Epotchs: 100
LR: 0.0032649444211699905
Optim: 0
Activat: 4
Init: 2
Regul: 0
DropNeed: 0
Score: 0.04337747022509575
Trial summary
Hyperparameters:
FirstHLNumber: 30
LayNumber: 8
Configuration: 5
BatchSize: 128
Epotchs: 100
LR: 0.002215807058277498
Optim: 6
Activat: 3
Init: 5
Regul: 0
DropNeed: 0
Score: 0.04483447223901749
Trial summary
Hyperparameters:
FirstHLNumber: 200
LayNumber: 4
Configuration: 3
BatchSize: 8
Epotchs: 100
LR: 0.0004274730816217117
Optim: 2
Activat: 3
Init: 1
Regul: 0
DropNeed: 0
Score: 0.04647070914506912
Trial summary
Hyperparameters:
FirstHLNumber: 100
LayNumber: 4
Configuration: 4
BatchSize: 8
Epo

In [13]:
best_hyperparameters = tuner.get_best_hyperparameters(Max_trials)
print('---------------BEST models --------------------------------------')
for i in range(0, Max_best_model):
    BestParametersToDNNandPrint(best_hyperparameters[i],
                                'Model rating: %02d' %(i+1))
    print('---------------------------------------------------------')
print('---------------------------------------------------------') 
print('---------------WORST models --------------------------------------')
for i in range(Max_trials-Max_best_model, Max_trials):
    BestParametersToDNNandPrint(best_hyperparameters[i],
                                'Model rating: %02d' %(i+1))
    print('---------------------------------------------------------')

---------------BEST models --------------------------------------
Model rating: 01
#Input Nodes = 4
ConfigurationType - Butterfly
[75, 67, 59, 51, 43, 35, 35, 43, 51, 59, 67, 75]
Batch Size = 8
# Epochs = 100
Learning Rate = 5.2712507092710046e-05
Ortimizer = Adam
Activation = elu
Initializer = XavierNormal
Regularizer = None
Dropout is not used
---------------------------------------------------------
Model rating: 02
#Input Nodes = 4
ConfigurationType - Triangle
[10, 9, 8, 7, 6, 5, 4, 3, 2, 1]
Batch Size = 16
# Epochs = 100
Learning Rate = 0.0032649444211699905
Ortimizer = Adam
Activation = elu
Initializer = HeNormal
Regularizer = None
Dropout is not used
---------------------------------------------------------
Model rating: 03
#Input Nodes = 4
ConfigurationType - Fir
[30, 27, 24, 21, 18, 15, 30, 27, 24, 21, 18, 15]
Batch Size = 128
# Epochs = 100
Learning Rate = 0.002215807058277498
Ortimizer = Nadam
Activation = selu
Initializer = RandomUniform
Regularizer = None
Dropout is not us

In [ ]:
# tuner.search_space_summary()
# tuner.results_summary


In [ ]:
# models = tuner.get_best_models(num_models = 2)
# for model in models:
#     model.summary()
#     model.evaluate(XtestMixt,YtestMixt)

In [ ]:
# tuner.oracle.get_best_trials(num_trials=3)[2].hyperparameters.values

In [ ]:
# Dnn.FirstHLNumber=25
# Dnn.LayersNumber=10
# Dnn.BS = 16
# Dnn.LRate = 1e-5
# Dnn.Epoch = 800
# Dnn.RegRate = 1e-7
# Dnn.DropRate = 0.25
# Dnn.SetConfigurationTypeByNumber(5)
# Dnn.SetOptimizerFunByNumber(3)
# Dnn.SetActivationFunByNumber(3)
# Dnn.SetInitializerTypeByNumber(2)
# Dnn.RegularizerTypeByNumber(1)
# Dnn.DropoutNeeded=True
# print(Dnn)